In [62]:
import sqlite3
import random
import pandas as pd
import glob
import os
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import wkt
import folium

In [47]:
def latlonTable2GDF(table: pd.DataFrame, colName_lat: str, colName_lon: str):
    return gpd.GeoDataFrame(table, geometry=gpd.points_from_xy(table[colName_lon], table[colName_lat])).set_crs(6668)

In [22]:
path_to_db = "./CityData.db"
path_to_pop_data = "../../../cityScope/data/sp_data_14100.csv"

In [3]:
conn = sqlite3.connect(path_to_db)
cursor = conn.cursor()
# cursor.execute("DROP TABLE IF EXISTS agent_locations")
cursor.execute("""CREATE TABLE IF NOT EXISTS agent_locations ( 
                timestamp INTEGER,
                person_name TEXT,
                gender TEXT,
                age INTEGER,
                latitude REAL,
                longitude REAL
                )""")
conn.commit()
conn.close()

In [68]:
conn = sqlite3.connect(path_to_db)
data.to_sql(name="agent_locations",con=conn,if_exists="append",index=False)
conn.commit()
conn.close()

In [79]:
conn = sqlite3.connect(path_to_db)
cursor = conn.cursor()
cursor.execute("SELECT * FROM agent_locations")
for r in cursor.fetchall():
    print(r)
conn.close()

In [23]:
df = pd.read_csv(path_to_pop_data)

In [24]:
df

,Unnamed: 0,age,sex_id,role_household_type_id,latitude,longitude,employment_type_id,company_id,industry_type_id,household_id
0,0,15,0,0,35.536595,139.657303,NaN,NaN,NaN,19438213
1,1,15,0,0,35.492273,139.661276,NaN,NaN,NaN,19438214
2,2,15,0,0,35.529866,139.670720,NaN,NaN,NaN,19438215
3,3,15,0,0,35.519671,139.682110,NaN,NaN,NaN,19438216
4,4,15,0,0,35.503321,139.672198,NaN,NaN,NaN,19438217
...,...,...,...,...,...,...,...,...,...,...
3495686,3495686,26,0,20,35.527540,139.587707,10.0,63.0,180.0,21007197
3495687,3495687,21,1,21,35.527540,139.587707,NaN,NaN,NaN,21007197
3495688,3495688,26,0,20,35.527540,139.587707,10.0,13.0,50.0,21007197
3495689,3495689,30,0,20,35.527540,139.587707,NaN,NaN,NaN,21007197


基本単位区データの取得

In [28]:
# 対象地区（横浜市）
target_jcodes = [
    "14101",
    "14102",
    "14103",
    "14104",
    "14105",
    "14106",
    "14107",
    "14108",
    "14109",
    "14110",
    "14111",
    "14112",
    "14113",
    "14114",
    "14115",
    "14116",
    "14117",
    "14118"
]

In [50]:
dirs = []
for dir in glob.glob("Z:/lab/accessibility/基本単位区/*.csv"):
    if os.path.basename(dir).split(".")[0] in target_jcodes:
        dirs.append(dir)
dfs = []
for path in dirs:
    dfs.append(pd.read_csv(path,usecols=["jcode","jinko","latitude","longitude"]))
df_pop = pd.concat(dfs,axis=0)

'01101.csv'

In [58]:
conn = sqlite3.connect("./tile.db")
cursor = conn.cursor()
# cursor.execute("""
#     SELECT * FROM tile
# """)
# for r in cursor.fetchall():
#     print(r)
df_tile = pd.read_sql("SELECT * FROM tile", conn)
conn.close()

In [59]:
df_tile

,zoom,x,y,x_rel,y_rel,geometry,label
0,17,116357,51727,0,0,"POLYGON ((139.584045 35.427106, 139.586792 35....",0
1,17,116357,51728,0,1,"POLYGON ((139.584045 35.424868, 139.586792 35....",0
2,17,116357,51729,0,2,"POLYGON ((139.584045 35.42263, 139.586792 35.4...",0
3,17,116357,51730,0,3,"POLYGON ((139.584045 35.420392, 139.586792 35....",0
4,17,116357,51731,0,4,"POLYGON ((139.584045 35.418153, 139.586792 35....",0
...,...,...,...,...,...,...,...
95,17,116366,51732,9,5,"POLYGON ((139.608765 35.415915, 139.611511 35....",0
96,17,116366,51733,9,6,"POLYGON ((139.608765 35.413677, 139.611511 35....",0
97,17,116366,51734,9,7,"POLYGON ((139.608765 35.411438, 139.611511 35....",0
98,17,116366,51735,9,8,"POLYGON ((139.608765 35.4092, 139.611511 35.40...",0


In [60]:
# tileで基本単位区からclip
gdf_pop = latlonTable2GDF(df_pop,"latitude","longitude")
df_tile['geometry'] = df_tile['geometry'].apply(wkt.loads)
gdf_tile = gpd.GeoDataFrame(df_tile,geometry="geometry")
gpd.sjoin(gdf_pop,gdf_tile,predicate="intersects")

C:\Users\tora2\AppData\Local\Temp\ipykernel_4560\2624089633.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:6668
Right CRS: None

  gpd.sjoin(gdf_pop,gdf_tile,predicate="intersects")


,jcode,jinko,latitude,longitude,geometry,index_right,zoom,x,y,x_rel,y_rel,label
644,14105,116,35.427063,139.610087,POINT (139.61009 35.42706),90,17,116366,51727,9,0,0
645,14105,87,35.426660,139.611444,POINT (139.61144 35.42666),90,17,116366,51727,9,0,0
664,14105,80,35.426303,139.611448,POINT (139.61145 35.4263),90,17,116366,51727,9,0,0
665,14105,44,35.425426,139.611257,POINT (139.61126 35.42543),90,17,116366,51727,9,0,0
666,14105,36,35.425651,139.610368,POINT (139.61037 35.42565),90,17,116366,51727,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3327,14111,155,35.405733,139.588956,POINT (139.58896 35.40573),19,17,116358,51736,1,9,0
3328,14111,32,35.405025,139.588659,POINT (139.58866 35.40502),19,17,116358,51736,1,9,0
3329,14111,14,35.404790,139.588882,POINT (139.58888 35.40479),19,17,116358,51736,1,9,0
3331,14111,109,35.404923,139.589786,POINT (139.58979 35.40492),29,17,116359,51736,2,9,0


In [55]:
gdf_pop

,jcode,jinko,latitude,longitude,geometry
0,14101,66,35.509479,139.681196,POINT (139.6812 35.50948)
1,14101,108,35.509366,139.680832,POINT (139.68083 35.50937)
2,14101,105,35.509113,139.681050,POINT (139.68105 35.50911)
3,14101,122,35.509112,139.681781,POINT (139.68178 35.50911)
4,14101,104,35.509611,139.680961,POINT (139.68096 35.50961)
...,...,...,...,...,...
3278,14118,8,35.533873,139.583445,POINT (139.58344 35.53387)
3279,14118,34,35.559950,139.591413,POINT (139.59141 35.55995)
3280,14118,69,35.552902,139.610380,POINT (139.61038 35.5529)
3281,14118,34,35.520656,139.557058,POINT (139.55706 35.52066)


In [56]:
gdf_tile

,zoom,x,y,x_rel,y_rel,geometry,label
0,17,116357,51727,0,0,"POLYGON ((139.58404 35.42711, 139.58679 35.427...",0
1,17,116357,51728,0,1,"POLYGON ((139.58404 35.42487, 139.58679 35.424...",0
2,17,116357,51729,0,2,"POLYGON ((139.58404 35.42263, 139.58679 35.422...",0
3,17,116357,51730,0,3,"POLYGON ((139.58404 35.42039, 139.58679 35.420...",0
4,17,116357,51731,0,4,"POLYGON ((139.58404 35.41815, 139.58679 35.418...",0
...,...,...,...,...,...,...,...
95,17,116366,51732,9,5,"POLYGON ((139.60876 35.41592, 139.61151 35.415...",0
96,17,116366,51733,9,6,"POLYGON ((139.60876 35.41368, 139.61151 35.413...",0
97,17,116366,51734,9,7,"POLYGON ((139.60876 35.41144, 139.61151 35.411...",0
98,17,116366,51735,9,8,"POLYGON ((139.60876 35.4092, 139.61151 35.4092...",0


ImportError: The matplotlib package is required for plotting in geopandas. You can install it using 'conda install -c conda-forge matplotlib' or 'pip install matplotlib'.

In [ ]:
# foliumマップの作成
m = folium.Map(location=[35.427106, 139.584045], zoom_start=13)

# GeoDataFrameをGeoJsonとして追加
folium.GeoJson(
    gdf,
    name='geojson',
    style_function=lambda x: {
        'color': 'blue',
        'weight': 2,
        'fillColor': 'cyan',
        'fillOpacity': 0.3
    }
).add_to(m)

# マップの表示
m